# UCN LD2 Calculations: Helical Groove

K. Augusto, J. Martin

June 1/20

Calculates the performance of a helical groove heat exchanger in the steady state operation of the LD2 thermosyphon using helical groove correlations for one groove.

In [1]:
#!/usr/bin/python3.7
from math import *
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from numpy import *

In [2]:
##############################################################
#General Constants Used
#############################################################
g=9.8 #m/s
p_psi16=20 # PSI, from CM.LD2.015.R2
p16=p_psi16*6894.76 # Pa
mu=3.5e-5 #Pa*s from https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote641.pdf
kt=0.104 #W/m*K from https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote641.pdf
beta_t=.012 # K^(-1), relative slope of density with temperature (a la Boussinesq)
cp=6565. # J/kg*K (for liquid D2), from CoolProp
rho=163 #kg/m3, from CRYOGENIC DATA BOOK by Dudley B. Chelton and Douglas B. Mann, pg 22
T=Tin=23.4 # (K) inlet temp
Tw=20.7 # (K) temperature of cold wall
mdot=0.004 # kg/s, from CM.LD2.015.R2
L=10*0.0254 #m length of tube
Cp=6565.4 # J/(kg*K), from CoolProp

##############################################################
#Geometry of HEX
#############################################################
Ngrooves=2 # number of grooves
D=4.76*0.0254 #m, from CM.LD2.015.R2
R=D/2
wprime= 0.015 #m width of groove
uprime= 0.01 # m width between grooves
depth=0.015 # m depth of groove
sinalpha=(Ngrooves*(wprime + uprime))/(pi*D) #pitch angle
alpha=arcsin(sinalpha)
print('The pitch angle is %f' %sinalpha)
print('Alpha is %f' %alpha)
Lprime=L/sinalpha #m length of wound groove
print('The length of the groove is %f m.' %Lprime)
turns=Lprime/(pi*D)
print('Coiling around a Cu rod of diameter %f m would require %f turns'%(D,turns))
#based off of sketch w/ jeff
w=wprime/sinalpha # m
ahelix=Ngrooves*wprime*depth #Arect+2*Atri ?? m^2 area of one helical groove/fin thing
print('The area of the helical fins is %f m^2.'%ahelix)
phelix=Ngrooves*(2*depth+2*wprime) #m
print('The perimeter of the helical grooves is %f m.' %phelix)
Dh=4*ahelix/phelix #m
print('Hydraulic diameter %f m'%Dh)
G=mdot/ahelix # (kg/(m^2*s)) mass flow rate per unit area
print('Mass flux per area (G) is %f kg/(m^2*s)'%G)

The pitch angle is 0.131637
Alpha is 0.132021
The length of the groove is 1.929542 m.
Coiling around a Cu rod of diameter 0.120904 m would require 5.080000 turns
The area of the helical fins is 0.000450 m^2.
The perimeter of the helical grooves is 0.120000 m.
Hydraulic diameter 0.015000 m
Mass flux per area (G) is 8.888889 kg/(m^2*s)


The Reynolds Number from Equation 4.10 in Sadik:

\begin{equation}
Re = \frac{4\dot{m}}{ \pi D \mu}
\end{equation}

In [3]:
Re=Dh*G/mu # should be dimensionless
print('The Reynolds number is %f'%Re)

The Reynolds number is 3809.523810


To characterize the flow in the helical groove, the Dean number is to be calculated. From the equation 3.47 of "HEAT EXCHANGERS: Selection, Rating, and Thermal Design" by Sadik Kakaç, Hongtan Liu, and Anchasa Pramuanjaroenkij, (Sadik), the Dean number is found by:

\begin{equation}
De = Re \sqrt{\frac{D_h}{D}}
\end{equation}

In [4]:
De=Re*sqrt(Dh/D)
print('The Deans number is %f.' %De)

The Deans number is 1341.825334.


Since the Dean number is greater than 300, and the Reynolds number is greater than 3500, the flow is turbulent. The friction factor is found from Equation 4.30 in Sadik,

\begin{equation}
f = 4fc\bigg(0.00725 + 0.076\bigg(Re\bigg(\frac{D}{D_h}\bigg)^{-2}\bigg)^{-0.25}\bigg)\left(\frac{D_h}{D}\right)^{1/2}
\end{equation}

In [5]:
f = 0.00725 + 0.076*(Re*(D/Dh)**(-2))**(-0.25)
fc = 4*f*(Dh/D)**(1/2)
print('The fricition factor is %f'%fc)

The fricition factor is 0.048910


The friction factor is found from Equation 4.31 in Sadik,

\begin{equation}
f_2 = 4fc_2\bigg(0.084\bigg(Re\bigg(\frac{D}{D_h}\bigg)^{-2}\bigg)^{-0.2}\bigg)\left(\frac{D_h}{D}\right)^{1/2}
\end{equation}

In [6]:
Cpw=5974.156073133989
muw=3.68e-5
Pr=Prw=(muw*Cpw)/(kt)
Prb=(mu*Cp)/kt
f2 = 0.084*(Re*(D/Dh)**(-2))**(-0.2)
fc2 = 4*f2*(Dh/D)**(1/2)
print('The second friction factor is %f'%fc2)

The second friction factor is 0.052424


As shown, the two friction factors are in goood agreement with each other.

The Nusselt number is found from the ratio of the coiled Nusselt number over the Nusselt number for a straight tube with the same dimensions as the helical tube. From equation 3.53 in Sadik,

\begin{equation}
Nu_c = \bigg(1 + 3.4\left(\frac{D_h}{D}\right)\left(\frac{Pr_b}{Pr_w}\right)^{0.25}\bigg)(Nu_s) = \bigg(1 + 3.4\left(\frac{D_h}{D}\right)\left(\frac{Pr_b}{Pr_w}\right)^{0.25}\bigg)(jHRePr^{1/3})
\end{equation}

In [7]:
Nu = (1 + 3.4 * (Dh/D))*(Prb/Prw)**(0.25)
print('The ratio of coiled Nusselt number to straight tube Nusselt number is Nuc/Nus %f' %Nu)
B1=1.174*((3.7e-5)/(3.68e-5))**(0.14)
jh=0.023*Re**(-0.2)*B1
Pr=(mu*Cp)/(kt)
Nus = jh*Re*Pr**(1./3.)
print('The coiled Nusselt number is %f' %(Nu*Nus))

The ratio of coiled Nusselt number to straight tube Nusselt number is Nuc/Nus 1.437628
The coiled Nusselt number is 37.053894


The flow velocity is found by:

\begin{equation}
u = \frac{G}{\rho}
\end{equation}

In [8]:
u=G/rho #m/s
print('The flow velocity is %f m/s' %u)

The flow velocity is 0.054533 m/s


The pressure drop for the first friction factor is found by Equations 4.34 and 4.35 in Sadik,

\begin{equation}
\Delta P = \frac{fcL'\rho u^2}{2D_h}
\end{equation}

In [9]:
dp = (fc*Lprime*rho*u**2)/(2*Dh)
print('The pressure drop is %f Pa from the first friction factor' %dp)

The pressure drop is 1.524880 Pa from the first friction factor


The pressure drop for the second friction factor is,

\begin{equation}
\Delta P = \frac{f_2L'\rho u^2}{2D_h}
\end{equation}

In [10]:
dp2 = (fc2*Lprime*rho*u**2)/(2*Dh)
print('The pressure drop is %f Pa from the second friction factor' %dp2)

The pressure drop is 1.634446 Pa from the second friction factor


The heat transfer coefficient is found from Equation 3.3 in Sadik,

\begin{equation}
hc = \frac{Nu_ckt}{D_h}
\end{equation}

In [11]:
h = Nus*Nu*kt/Dh # (mu*Cp*Pr**(-2/3)*Re*0.023*Re**(-0.2))/(Dh)
print('The heat transfer coeff is %f W/m^2K.' %h)

The heat transfer coeff is 256.907000 W/m^2K.


The number of transfer units is found from Equation 6.41 of Barron,

\begin{equation}
Ntu = \frac{hA_w}{\dot{m}C_p}
\end{equation}

where the area of the cold wall is

\begin{equation}
A_w = N(w' + 2depth)L'
\end{equation}

In [12]:
Aw=Ngrooves*(wprime+2*depth)*Lprime
print('Area of cold wall %f m^2'%Aw)
Ntu=h*Aw/(mdot*Cp)
print('The number of transfer units is %f'%Ntu)
print()

Area of cold wall 0.173659 m^2
The number of transfer units is 1.698836



The total heat transfer is found from Equation 2.5 and 2.6 from Sadik,

\begin{equation}
Q = \dot{m}C_p (T_1-T_2)
\end{equation}

In [13]:
T1=Tin
T2=T1-(T1-Tw)*(1-exp(-Ntu))
T2=Tw+(T1-Tw)*exp(-Ntu)
Qtotal=mdot*Cp*(T1-T2) # Eq. (6.43) of Barron
print('For inlet temperature %f K and wall temperature %f K'%(T1,Tw))
print('the outlet temperature is %f K'%T2)
print('and the total heat transfer rate is %f W'%Qtotal)

For inlet temperature 23.400000 K and wall temperature 20.700000 K
the outlet temperature is 21.193820 K
and the total heat transfer rate is 57.937820 W
